In [1]:
import os
import pandas as pd
import openpyxl
import math

# Define the file to exclude
excluded_file = 'Sets.csv'

# Define the directory where you want to save the filtered CSV files
output_csv_directory = 'output_csv'  # Change this to the directory where you want to save the filtered CSV files

# Create the output directory for CSV files if it doesn't exist
os.makedirs(output_csv_directory, exist_ok=True)

# Define the directory where you want to save the Excel file
output_excel_directory = 'output_excel'  # Change this to the directory where you want to save the Excel file

# Create the output directory for the Excel file if it doesn't exist
os.makedirs(output_excel_directory, exist_ok=True)

# Specify the Excel file path
excel_file_path = 'GENeSYS-MOD_User_Input_Settings_v03_phe_06-09-2023.xlsx'  # Replace with the path to your Excel file

In [2]:
# Open the Excel file
xls = pd.ExcelFile(excel_file_path, engine='openpyxl')

# Get the list of sheet names in the Excel file
sheets_to_read = xls.sheet_names

# Initialize an empty dictionary to store DataFrames
data_frames = {}
filtered_df = {}
unique_values = {}

unique_values_concatenated = pd.DataFrame()
column_list = []

# Read sheets and store them in the dictionary
for sheet_name in sheets_to_read:
    data_frames = xls.parse(sheet_name)

    filtered_df= data_frames[data_frames.iloc[:, 1] == 1] # Assuming the second column is indexed at 1 (0-based index)

    column_list.append(filtered_df.columns[0]) # collect column header for each set sheet
   
    unique_values= pd.DataFrame(filtered_df.iloc[:, 0].unique())  # Assuming the first column is indexed at 0 (0-based index)
    unique_values_parameter = pd.DataFrame(unique_values)
    
    unique_values_concatenated = pd.concat([unique_values_concatenated, unique_values], axis=1)

# Need to put header to the dataframe
unique_values_concatenated.columns = column_list

# Create a CSV file containing unique values
unique_values_csv_file_path = os.path.join(output_csv_directory, 'Sets.csv')
unique_values_concatenated.to_csv(unique_values_csv_file_path, index=False, decimal='.') 


In [3]:
# Initialize an empty dictionary to store DataFrames
data_frames = {}


In [13]:
# Define the directories
current_directory = os.getcwd()
sets_and_tags_directory = os.path.join(current_directory, '00_Sets&Tags')

# Get a list of subdirectories in the current directory
subdirectories_current = [d for d in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, d)) and d.startswith("Par_")]

# Get list of csv files starting with Par_ from 00_Sets&Tags
par_csv_files_sets_and_tags = [f for f in os.listdir(sets_and_tags_directory) if f.startswith('Par_') and f.endswith('.csv')]

# For those CSV files, we'll treat their path as a "subdirectory" (even though they aren't directories)
par_csv_filepaths_sets_and_tags = [os.path.join(sets_and_tags_directory, f) for f in par_csv_files_sets_and_tags]

# Combine subdirectories from the current directory with filepaths from 00_Sets&Tags
all_paths = subdirectories_current + par_csv_filepaths_sets_and_tags


# Initialize the Excel writer
output_excel_file_path = os.path.join(output_excel_directory, 'output.xlsx')
with pd.ExcelWriter(output_excel_file_path, engine='openpyxl') as writer:
    unique_values_concatenated.to_excel(writer, sheet_name='Sets', index=False, header=True)
    
    # Process CSV files in each path
    for path in all_paths:
        if os.path.isdir(path):
            # If it's a directory, list all CSV files within
            csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
            csv_filepaths = [os.path.join(path, f) for f in csv_files]
        else:
            # If it's a CSV file, use it directly
            csv_filepaths = [path]
            

        for csv_file_path in csv_filepaths:
            # Read the CSV file into a Pandas DataFrame
            df = pd.read_csv(csv_file_path, delimiter=',')
            
            # Create a list of columns to keep
            columns_to_keep = [col for col in df.columns if col in unique_values_concatenated.columns or col == 'Value']
            
            # Filter the DataFrame to keep only the selected columns
            df = df[columns_to_keep]
            
            pivot_indices = [col for col in unique_values_concatenated.columns if col in df.columns]
        
            # Iterate over unique_values_concatenated DataFrame columns
            for header in unique_values_concatenated.columns:
                if header in df.columns:
                    # Filter the DataFrame based on whether the values in the header column are present in unique_values_concatenated
                    df = df[df[header].isin(unique_values_concatenated[header])]
            
            # Pivot after filtering
            if 'Year' in df.columns:
                cols_to_drop = [col for col in df.columns if col not in unique_values_concatenated.columns and col not in ['Year', 'Value']]
                df = df.drop(columns=cols_to_drop)
                df = df.pivot_table(index=pivot_indices, columns='Year', values='Value', aggfunc='first').reset_index()
                if 'Year' in df.columns:
                    df = df.drop(columns='Year')
   
            filtered_df = df    
    
            filtered_df = pd.concat([filtered_df.columns.to_frame().T, filtered_df], ignore_index=True)
            filtered_df.columns = range(len(df.columns))
        
            df_final = filtered_df
        
            df_final.replace('nan', '', inplace=True)
            df_final.apply(lambda x: x.apply(lambda y: str(y).replace('.', ',')))
        
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(os.path.basename(csv_file_path))[0]
        
            # Store the DataFrame in the dictionary with the filename (without extension) as the key
            data_frames[worksheet_name] = df_final
        
            # Construct the full path to the output CSV file
            output_csv_file_path = os.path.join(output_csv_directory, os.path.basename(csv_file_path))
        
            # Save the filtered DataFrame to the CSV file without the index
            df_final.to_csv(output_csv_file_path, index=False, header=False, decimal='.')
        
            # Save the filtered DataFrame to the Excel file without the index
            df_final.to_excel(writer, sheet_name=worksheet_name, index=False, header=False)

In [5]:
# Get a list of subdirectories in the current directory
subdirectories_TS = [d for d in os.listdir() if os.path.isdir(d) and d.startswith("TS_")]
# Initialize the Excel writer for hourly (TS_) files     
output_excel_file_path_TS = os.path.join(output_excel_directory, 'output_TS.xlsx')
with pd.ExcelWriter(output_excel_file_path_TS, engine='openpyxl') as writer:
    # Process CSV files in each subdirectory
    for subdirectory in subdirectories_TS:
        # Logic to read in the hourly files and exporting the data for GENeSYS-MOD as .csv or excel file
        csv_files = [f for f in os.listdir(subdirectory) if f.endswith('.csv')]
         
        for csv_file in csv_files:# Construct the full path to the CSV file
            csv_file_path = os.path.join(subdirectory, csv_file)
            
            df_TS = pd.read_csv(csv_file_path, delimiter=',', skiprows=[0])
            
            # Create a DataFrame containing only the "hour" column
            hour_column = df_TS["HOUR"].to_frame()

            #Create a list called "selected_regions" containing the unique values from the "Region" column
            selected_regions_TS = unique_values_concatenated["Region"].unique().tolist()

            selected_regions_TS = [value for value in selected_regions_TS if not isinstance(value, float) or not math.isnan(value)]

            #Filter the columns in the "TS_WIND_ONSHORE_INF.csv" DataFrame
            filtered_df_TS = df_TS[selected_regions_TS]

            # Concatenate the "hour" DataFrame as the first row of the filtered DataFrame
            filtered_df_TS = pd.concat([hour_column, filtered_df_TS], axis=1, ignore_index=False)
            
            df_TS_final = filtered_df_TS
            
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(csv_file)[0]
            
            # Store the DataFrame in the dictionary with the filename (without extension) as the key
            data_frames[worksheet_name] = df_TS_final
            
            
            # Write the DataFrame to an Excel worksheet with the same filename (without extension)
            df_TS_final.to_excel(writer, sheet_name=worksheet_name, index=False, header=True)
            
            
            # Specify the path where you want to save the CSV file
            output_csv_file_path = os.path.join(output_csv_directory, csv_file)

            # Use the to_csv method to save the DataFrame to a CSV file
            df_TS_final.to_csv(output_csv_file_path, index=False)
    